

1.   Importing libraries



In [ ]:
import os
import shutil

In [ ]:
from sklearn.model_selection import train_test_split



2.   Connecting to the Drive



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive




3.   Data preparation



In [ ]:
# Data directories in Drive
data_dir = '/content/drive/My Drive/contact-maps'
alpha_dir = os.path.join(data_dir, 'alpha-helices')
beta_dir = os.path.join(data_dir, 'beta-sheets')

In [ ]:
alpha_helices = [os.path.join(alpha_dir, f) for f in os.listdir(alpha_dir) if f.endswith('.png')]
beta_sheets = [os.path.join(beta_dir, f) for f in os.listdir(beta_dir) if f.endswith('.png')]

In [ ]:
alpha_train, alpha_val = train_test_split(alpha_helices, test_size=0.2, random_state=42)
beta_train, beta_val = train_test_split(beta_sheets, test_size=0.2, random_state=42)

In [ ]:
yolo_data_dir = '/content/yolo-contact-maps'
train_dir = os.path.join(yolo_data_dir, 'train/images')
val_dir = os.path.join(yolo_data_dir, 'val/images')
train_labels_dir = os.path.join(yolo_data_dir, 'train/labels')
val_labels_dir = os.path.join(yolo_data_dir, 'val/labels')

In [ ]:
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)



5.   Data preprocessing



In [ ]:
# Function to generate annotations for contact maps
def create_annotation(label_path, class_id, x_center=0.5, y_center=0.5, width=0.7, height=0.4):
    with open(label_path, 'w') as f:
        f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

In [ ]:
# Function for copying images
def prepare_data(img_files, dst_img_dir, dst_lbl_dir, class_id, width, height):
    for img_path in img_files:
        img_file = os.path.basename(img_path)
        dst_img_path = os.path.join(dst_img_dir, img_file)
        shutil.copy(img_path, dst_img_path)

        # Gerar anotação
        label_file = img_file.replace('.png', '.txt')
        label_path = os.path.join(dst_lbl_dir, label_file)
        create_annotation(label_path, class_id, width=width, height=height)

In [ ]:
# Copy training files
prepare_data(alpha_train, train_dir, train_labels_dir, class_id=0, width=0.7, height=0.4)
prepare_data(beta_train, train_dir, train_labels_dir, class_id=1, width=0.6, height=0.3)

# Copy validation files
prepare_data(alpha_val, val_dir, val_labels_dir, class_id=0, width=0.7, height=0.4)
prepare_data(beta_val, val_dir, val_labels_dir, class_id=1, width=0.6, height=0.3)



6.   Treinamento



In [ ]:
# YOLOv5 data file configuration
data_yaml = """
train: /content/yolo-contact-maps/train
val: /content/yolo-contact-maps/val

nc: 2
names: ['alpha-helix', 'beta-sheet']
"""

In [ ]:
with open(os.path.join(yolo_data_dir, 'data.yaml'), 'w') as f:
    f.write(data_yaml)

In [ ]:
# Clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 16725, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 16725 (delta 11), reused 32 (delta 7), pack-reused 16679
Receiving objects: 100% (16725/16725), 15.31 MiB | 15.22 MiB/s, done.
Resolving deltas: 100% (11490/11490), done.
/content/yolov5


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.7/793.7 kB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl

In [ ]:
# Train the YOLOv5 model with transfer learning
!python train.py --img 640 --batch 16 --epochs 50 --data /content/yolo-contact-maps/data.yaml --weights yolov5s.pt

2024-07-03 12:12:57.553967: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-03 12:12:57.554021: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-03 12:12:57.555511: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/yolo-contact-maps/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=Fa